In [1]:
# Package Imports
import pandas as pd
import numpy as np

import unicodedata
import string

import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

# Suppress pandas FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
### Functions to load Datasets ###
def import_train_csvs() -> pd.DataFrame:
    # Read train-X.csv, combine and return df 
    col_names = ['index','movie_id','primaryTitle','originalTitle','startYear','endYear','runtimeMinutes','numVotes','label']
    train_df = pd.DataFrame(columns=col_names)
    for i in range(1, 9):
        auxilliary_df = pd.read_csv(filepath_or_buffer=f'./imdb/train-{i}.csv',
                                    header=0,
                                    names=col_names,
                                    na_values='\\N')
        train_df = pd.concat([train_df, auxilliary_df])
     
    return train_df

def import_hidden_csv(filename: str) -> pd.DataFrame:
    # Read the validation-hidden.csv and return df
    col_names = ['index','movie_id','primaryTitle','originalTitle','startYear','endYear','runtimeMinutes','numVotes']
    hidden_df = pd.read_csv(filepath_or_buffer=f'./imdb/{filename}', 
                                header=0,
                                names=col_names,
                                na_values='\\N')
    return hidden_df

def import_directors() -> pd.DataFrame:
    # Read directors from .json file and return df
    directors_df = pd.read_json('./imdb/directing.json')
    directors_df.columns = ['movie_id', 'director_id']
    return directors_df

def import_writers() -> pd.DataFrame:
    # Read writers from .json file and return df
    writers_df = pd.read_json('./imdb/writing.json')
    writers_df.columns = ['movie_id', 'writer_id']
    return writers_df

def import_external_data() -> pd.DataFrame:
    # Read and combine .csv files from the external_data folder
    movie_categories = ['action', 'adventure', 'animation', 'biography', 'crime', 'family', 'fantasy', 'film-noir', 'history', 'horror', 'mystery', 'romance', 'scifi', 'sports', 'thriller', 'war']
    df = pd.DataFrame(columns=['movie_name', 'year', 'certificate', 'runtime', 'genre', 'rating','description', 'director',
                               'director_id', 'star', 'star_id', 'votes','gross(in $)'])
    for category in movie_categories:
        auxilliary_df = pd.read_csv(f'./imdb/extra/external_full/{category}.csv')
        df = pd.concat([df, auxilliary_df], ignore_index=True)
        df = df.drop_duplicates(subset=['movie_id'], keep='first')
        df = df.drop_duplicates(subset=['movie_name', 'runtime', 'year'], keep='first')
        
    return df
        

In [3]:
train_df = import_train_csvs()
train_df

,index,movie_id,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919.0,NaN,66.0,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920.0,NaN,145.0,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921.0,NaN,97.0,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924.0,NaN,59.0,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925.0,NaN,93.0,17887.0,True
...,...,...,...,...,...,...,...,...,...
988,9966,tt9625664,Trauma Center,NaN,2019.0,NaN,87.0,12951.0,False
989,9981,tt9741310,Slaxx,Slaxx,2020.0,NaN,77.0,2464.0,False
990,9982,tt9742392,Kindred,Kindred,2020.0,NaN,101.0,1719.0,False
991,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020.0,NaN,111.0,4144.0,True


In [4]:
### Functions to Preprocess Datasets ###
# Function to remove accents
def remove_accents(input_str):
    try:
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
    except:
        return input_str

# Function to remove punctuation
def remove_punctuation(input_str):
    try:
        return input_str.translate(str.maketrans('', '', string.punctuation))
    except:
        return input_str

def preprocess_train(df: pd.DataFrame, directors_df: pd.DataFrame, writers_df: pd.DataFrame) -> pd.DataFrame:
    # New releaseDate column
    df['releaseDate'] = df.apply(lambda row: row['endYear'] if row['startYear'] == "\\N" else row['startYear'], axis=1)
    
    # Drop unused columns
    df = df.drop(columns=["startYear", "endYear", 'index'])
    
    # Impute NA values in numVotes, runtimeMinutes and releaseDate columns
    median_numVotes = int(df['numVotes'].median())
    mean_runtimeMinutes = int(df['runtimeMinutes'].mean())
    median_releaseDate = int(df['releaseDate'].median())
    df['numVotes'] = df['numVotes'].fillna(median_numVotes)
    df['runtimeMinutes'] = df['runtimeMinutes'].fillna(mean_runtimeMinutes)
    df['releaseDate'] = df['releaseDate'].fillna(median_releaseDate)
    
    # encode True/False labels to 1/0 values
    df['label'] = df['label'].apply(lambda x: 1 if x else 0)
    
    # Remove accents and punctuation from primary titles
    df['primaryTitle'] = df['primaryTitle'].apply(remove_accents)
    
    # Original Title feature
    df['has_original_title'] = df['originalTitle'].str.contains('NaN')
    df['has_original_title'] = df['has_original_title'].apply(lambda x: 0 if x else 1)
    df = df.drop(columns=["originalTitle"])
    
    # Set correct dtypes
    df['releaseDate'] = df['releaseDate'].astype('int')
    df['runtimeMinutes'] = df['runtimeMinutes'].astype('int')
    
    # Add number of writers & directors
    # Count the number of directors & writers per movie
    directors_per_movie = directors_df.groupby('movie_id')['director_id'].nunique().reset_index(name='num_directors')
    writers_per_movie = writers_df.groupby('movie_id')['writer_id'].nunique().reset_index(name='num_writers')
    
    # Merge director counts and writer counts with train_df
    df = df.merge(directors_per_movie, on='movie_id', how='left')
    df = df.merge(writers_per_movie, on='movie_id', how='left')
    
    return df

def preprocess_hidden(df: pd.DataFrame, 
                      directors_df: pd.DataFrame, writers_df: pd.DataFrame, 
                      median_num_votes: int, mean_runtime_minutes: int, median_release_date: int = None
                      ) -> pd.DataFrame:
    # New releaseDate column
    df['releaseDate'] = df.apply(lambda row: row['endYear'] if row['startYear'] == "\\N" else row['startYear'], axis=1)

    # Drop unused columns
    df = df.drop(columns=["startYear", "endYear", 'index'])
    
    # Impute NA values in numVotes, runtimeMinutes and releaseDate columns
    df['numVotes'] = df['numVotes'].fillna(median_num_votes)
    df['runtimeMinutes'] = df['runtimeMinutes'].fillna(mean_runtime_minutes)
    df['releaseDate'] = df['releaseDate'].fillna(median_release_date)
    
    # Remove accents and punctuation from primary titles
    df['primaryTitle'] = df['primaryTitle'].apply(remove_accents)
    
    # Original Title feature
    df['has_original_title'] = df['originalTitle'].str.contains('NaN')
    df['has_original_title'] = df['has_original_title'].apply(lambda x: 0 if x else 1)
    df = df.drop(columns=["originalTitle"])
    
    # Add number of writers & directors
    # Count the number of directors & writers per movie
    directors_per_movie = directors_df.groupby('movie_id')['director_id'].nunique().reset_index(name='num_directors')
    writers_per_movie = writers_df.groupby('movie_id')['writer_id'].nunique().reset_index(name='num_writers')
    
    # Merge director counts and writer counts with train_df
    df = df.merge(directors_per_movie, on='movie_id', how='left')
    df = df.merge(writers_per_movie, on='movie_id', how='left')
    
    return df

def preprocess_external_data(df: pd.DataFrame) -> pd.DataFrame:
    # Remove accents from movie_name column
    df['movie_name'] = df['movie_name'].apply(remove_accents)
    
    # Turn Runtime to int column
    df['runtime'] = df['runtime'].str.extract(r'(\d+)')
    
    # Turn Year into int column
    df['year'] = df['year'].str.extract(r'(\d+)')
    
    # Create boolean genre columns
    movie_genres = ['action', 'adventure', 'animation', 'biography', 'crime', 'family', 'fantasy', 'film-noir', 'history', 'horror', 'mystery', 'romance', 'scifi', 'sports', 'thriller', 'war', 'comedy']
    
    df['genre'] = df['genre'].str.lower()
    for genre in movie_genres:
        col_name = 'genre_' + genre
        df[col_name] = df['genre'].str.contains(genre)
        df[col_name] = df[col_name].apply(lambda x: 1 if x else 0)
        
    # Create boolean rating columns
    movie_ratings = ['Not Rated', 'R', 'PG-13', 'Passed', 'Approved']
    for rating in movie_ratings:
        col_name = 'movie_rating_' + rating
        df[col_name] = df['genre'].str.contains(rating)
        df[col_name] = df[col_name].apply(lambda x: 1 if x else 0)
    
    # Set column types
    df = df.astype({'year': 'Int64', 'runtime': 'Int64'})
    
    # Drop un-used columns
    df = df.drop(columns=['movie_id', 'rating', 'description', 'director', 'director_id', 'star', 'star_id', 'genre', 'certificate'])
    
    return df
    

In [5]:
### XGBoost Script ###
# Load Datasets 
train_df = import_train_csvs()
validation_df = import_hidden_csv(filename='validation_hidden.csv')
test_df = import_hidden_csv(filename='test_hidden.csv')
directors_df = import_directors()
writers_df = import_writers()
external_df = import_external_data()

# Preprocess Datasets
train_df = preprocess_train(train_df, directors_df, writers_df)
validation_df = preprocess_hidden(df=validation_df,
                                  directors_df=directors_df,
                                  writers_df=writers_df,
                                  median_num_votes=train_df['numVotes'].median(),
                                  mean_runtime_minutes=train_df['runtimeMinutes'].mean(),
                                  median_release_date=train_df['releaseDate'].median())
test_df = preprocess_hidden(df=test_df,
                            directors_df=directors_df,
                            writers_df=writers_df,
                            median_num_votes=train_df['numVotes'].median(),
                            mean_runtime_minutes=train_df['runtimeMinutes'].mean(),
                            median_release_date=train_df['releaseDate'].median())
external_df = preprocess_external_data(df=external_df)

# Merge external dataset with train, validation and test datasets
print(train_df.shape)
print(validation_df.shape)
print(test_df.shape)
train_df = train_df.merge(external_df, 
                          #on='movie_id',
                          left_on=['primaryTitle', 'releaseDate', 'runtimeMinutes'], 
                          right_on=['movie_name', 'year', 'runtime'],
                          how='left')
validation_df = validation_df.merge(external_df, 
                                    #on='movie_id',
                                    left_on=['primaryTitle', 'releaseDate', 'runtimeMinutes'], 
                                    right_on=['movie_name', 'year', 'runtime'],
                                    how='left')
test_df = test_df.merge(external_df, 
                        #on='movie_id', 
                        left_on=['primaryTitle', 'releaseDate', 'runtimeMinutes'], 
                        right_on=['movie_name', 'year', 'runtime'],
                        how='left')

print(train_df.shape)
print(validation_df.shape)
print(test_df.shape)

(7959, 9)
(955, 8)
(1086, 8)
(7959, 36)
(955, 35)
(1086, 35)


/var/folders/pv/ng4pj3dx1lzdf21rlczf5b900000gn/T/ipykernel_40072/3654897382.py:35: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  validation_df = validation_df.merge(external_df,
/var/folders/pv/ng4pj3dx1lzdf21rlczf5b900000gn/T/ipykernel_40072/3654897382.py:40: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  test_df = test_df.merge(external_df,


In [6]:
### XGBoost ###
# Selecting X and y from train_df
X = train_df.drop(columns=['movie_id', 'primaryTitle', 'label', 'movie_name'])
y = train_df['label']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Define hyperparameters grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.15, 0.2]
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(best_params)
# Instantiate XGBoost with the best parameters
best_xgb_model = xgb.XGBClassifier(**best_params)

# Train the model with the best parameters
best_xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = best_xgb_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 200}
Accuracy: 0.7983668341708543


In [7]:
validation_df = validation_df.drop(columns=['movie_id', 'primaryTitle', 'movie_name'])
test_df = test_df.drop(columns=['movie_id', 'primaryTitle', 'movie_name'])

In [8]:
# Predict on validation set with XGBoost
val_pred = best_xgb_model.predict(validation_df)

# Predict on test set with XGBoost
test_pred = best_xgb_model.predict(test_df)

In [9]:
# Turn predictions into booleans
to_bool = lambda val: val == 1
val_pred = np.array([to_bool(i) for i in val_pred])
test_pred = np.array([to_bool(i) for i in test_pred])

# Export predictions to .csv files
pd.DataFrame(val_pred).to_csv('val_pred.csv', index=False, header=False)
pd.DataFrame(test_pred).to_csv('test_pred.csv', index=False, header=False)

In [19]:
# Compare pandas and pyspark predictions
# Filepaths
pyspark_val_pred_path = './bd-imdb/predictions_val_pred/val_pred.csv'
pyspark_test_pred_path = './bd-imdb/predictions_test_pred/test_pred.csv'

# Create dataframes
pandas_val_pred = pd.DataFrame(val_pred, columns=['label'])
pandas_test_pred = pd.DataFrame(test_pred, columns = ['label'])
pyspark_val_pred = pd.read_csv(pyspark_val_pred_path, header=None, names=["label"])
pyspark_test_pred = pd.read_csv(pyspark_test_pred_path, header=None, names=["label"])
print(pandas_test_pred.shape)
print(pyspark_test_pred.shape)
# Calculate share of observations
validation_match = (pandas_val_pred['label'] == pyspark_val_pred['label']).sum() / len(pandas_val_pred)
testing_match = (pandas_test_pred['label'] == pyspark_test_pred['label']).sum() / len(pandas_test_pred)
print(validation_match)
print(testing_match)

(1086, 1)
(1086, 1)
0.8722513089005236
0.8554327808471455


In [18]:
external = pd.read_csv('./bd-imdb/imdb/extra/external.csv')
external['star'] = external['star'].str.replace('')

,movie_id,movie_name,year,certificate,runtime,genre,rating,star,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...",NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,"Sam Worthington, \nZoe Saldana, \nSigourney We...",NaN
2,tt5884796,Plane,2023,R,107 min,"Action, Thriller",6.5,"Gerard Butler, \nMike Colter, \nTony Goldwyn, ...",NaN
3,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...",NaN
4,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...",NaN
...,...,...,...,...,...,...,...,...,...
243192,tt0043126,Western Pacific Agent,1950,NaN,65 min,"Drama, Film-Noir",5.9,"Kent Taylor, \nSheila Ryan, \nMickey Knox, \nM...",NaN
243193,tt0049171,Edge of Hell,1956,NaN,78 min,"Drama, Film-Noir",5.6,"Hugo Haas, \nFrancesca De Scaffa, \nJune Shell...",NaN
243194,tt0039742,Que Dios me perdone,1948,NaN,97 min,"Drama, Film-Noir",6.2,"María Félix, \nFernando Soler, \nJulián Soler,...",NaN
243195,tt0030627,Prison Nurse,1938,NaN,67 min,"Drama, Film-Noir",5.6,"Henry Wilcoxon, \nMarian Marsh, \nBernadene Ha...",NaN
